In [39]:
import scipy.stats as stats
import math
import nltk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import unicodedata
import string
import os
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from collections import Counter

In [2]:
vectors = np.load('tfidf.npy')

In [6]:
X_train = pd.read_json('X_train.json', orient = 'table')

In [7]:
y_train = pd.read_json('y_train.json', orient = 'table')

In [5]:
vectors.shape

(19965, 3586)

In [6]:
veclist = list(vectors)

In [7]:
len(veclist[1])

3586

In [8]:
X_train['tfidf'] = veclist

In [9]:
yt = list(y_train['price'])

In [10]:
rf = RandomForestRegressor(n_estimators = 50, max_features = 'auto', random_state = 0)

In [11]:
rf.fit(veclist, yt)

RandomForestRegressor(n_estimators=50, random_state=0)

In [2]:
from joblib import dump, load
rf = load('rf.sav')

In [3]:
def removeaccents(text):
    nfkdform = unicodedata.normalize('NFKD', text)
    asctext = nfkdform.encode('ASCII', 'ignore')
    return asctext.decode()

In [4]:
def filter_tokens(tokens):
    return([w for w in tokens if not w in stopwords_ and not w in punctuation_])

In [8]:
    termslist = []
    corpusvocab = set()
    paragraphs = X_train['roomstext']
    documents = []
    for paragraph in paragraphs:
        documents.append(str(paragraph))
    texts = []
    for document in documents:
        soup = BeautifulSoup(document, 'html.parser')
        texts.append(soup.get_text())
    
    ratexts = []
    for text in texts:
        txt = removeaccents(text)
        ratexts.append(txt)

    
    for text in ratexts:
        senttokens = nltk.tokenize.sent_tokenize(text)
        tokens = [sent for sent in map(nltk.tokenize.word_tokenize, senttokens)]
        lower = [[word.lower() for word in sent] for sent in tokens]
        stopwords_ = set(nltk.corpus.stopwords.words('english'))
        stopwords_.discard('not')
        punctuation_ = set(string.punctuation)
        filteredtokens = list(map(filter_tokens, lower))
        stemporter = nltk.stem.porter.PorterStemmer()
        stemtokens = [list(map(stemporter.stem, word))for word in filteredtokens]
        terms = []
        vocab = set()
        for stems in stemtokens:
            for word in stems:
                vocab.add(word)
                terms.append(word)
        termslist.append(terms)
        corpusvocab.update(vocab)

In [9]:
dococcur = Counter( [word for term in termslist for word in set(term)])

In [10]:
docfreq = {k:(v/float(len(termslist))) for k,v in dococcur.items() }

In [11]:
mindf = .002
vocab = [k for k, v in docfreq.items() if v > mindf]

In [12]:
len(vocab)

3586

In [13]:
X_test = pd.read_json('X_test.json', orient = 'table')

In [14]:
    termslistt = []
    corpusvocab = set()
    paragraphs = X_test['roomstext']
    documents = []
    for paragraph in paragraphs:
        documents.append(str(paragraph))
    texts = []
    for document in documents:
        soup = BeautifulSoup(document, 'html.parser')
        texts.append(soup.get_text())
    
    ratexts = []
    for text in texts:
        txt = removeaccents(text)
        ratexts.append(txt)

    
    for text in ratexts:
        senttokens = nltk.tokenize.sent_tokenize(text)
        tokens = [sent for sent in map(nltk.tokenize.word_tokenize, senttokens)]
        lower = [[word.lower() for word in sent] for sent in tokens]
        stopwords_ = set(nltk.corpus.stopwords.words('english'))
        stopwords_.discard('not')
        punctuation_ = set(string.punctuation)
        filteredtokens = list(map(filter_tokens, lower))
        stemporter = nltk.stem.porter.PorterStemmer()
        stemtokens = [list(map(stemporter.stem, word))for word in filteredtokens]
        terms = []
        tempvocab = set()
        for stems in stemtokens:
            for word in stems:
                tempvocab.add(word)
                terms.append(word)
        termslistt.append(terms)
        corpusvocab.update(tempvocab)

In [15]:
y_test = pd.read_json('y_test.json', orient = 'table')

In [16]:
len(vocab)

3586

In [17]:
len(corpusvocab)

38558

In [18]:
vectorst = np.zeros((len(termslistt), len(vocab)))

In [19]:
vectorst.shape

(6655, 3586)

In [20]:
termoccur = list(map(lambda term : Counter(term), termslistt))

In [21]:
termfreq = []
for i in range(len(termoccur)):
    termfreq.append({k : (v / float(len(termslistt[i]))) 
                    for k, v in termoccur[i].items()})

In [22]:
for i in range(len(termslistt)):
    for j in range(len(vocab)):
        tempterm = vocab[j]
        temptermf = termfreq[i].get(tempterm, 0.0)
        termidf = np.log(1+1/docfreq[tempterm])
        vectorst[i,j] = temptermf * termidf

In [35]:
ytest = np.array(y_test['price'])

In [24]:
veclistt = list(vectorst)

In [25]:
pred = rf.predict(veclistt)


In [33]:
type(pred)

numpy.ndarray

In [45]:
len(ytest)

6655

In [49]:
summ = 0
for i in range(len(ytest)):
    summ +=(ytest[i]-pred[i])**2
result = summ**.5/len(ytest)

In [50]:
result

63.85014515096804

In [52]:
base = np.full(6655,np.mean(ytest))

In [54]:
summ = 0
for i in range(len(ytest)):
    summ +=(ytest[i]-base[i])**2
result = summ**.5/len(ytest)

In [55]:
result

58.670168164490875